In [96]:
import numpy as np
from keras.datasets import mnist

In [97]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [98]:
x_train.shape

(60000, 28, 28)

In [99]:
images = x_train[:10000].reshape(10000, 28*28)
labels = y_train[0:10000].reshape(10000,1)

In [100]:
def preparing_data(X):
    mean_x = np.mean(X)
    std = np.sum(X**2)/(X.shape[0]-1)
    return (X - mean_x)/std

In [101]:
images = preparing_data(images)

In [102]:
from neural_network import NeuralNetwork

In [124]:
nn = NeuralNetwork()
nn.CreateLayerRelu((count_of_features, 1000))
nn.CreateLayerRelu((1000, 500))
nn.CreateLayerLinear((500, count_classes))

In [125]:
count_of_features = images.shape[1]
count_classes = 10
batch_size = 500
learing_rate = 0.001

In [126]:
epochs = 300
loss = 0
for it in range(epochs):
    batch_indexes = np.random.randint(0, images.shape[0], batch_size)
    batch_X = images[batch_indexes]
    batch_y = labels[batch_indexes]
    output = nn.forward(batch_X)
    prediction = nn.softmax(output)
    loss = nn.CrossEntropyLoss(prediction, batch_y)
    nn.backward(learing_rate)
    print(loss)

26.454469028663866
30.607563728779258
31.13485571096006
24.827169507786436
14.901575873015423
11.104204811400804
8.016312903343767
3.354593307931821
2.0441313334015523
2.1293571847257664
1.531552054306428
1.62721452233404
1.6860157890155707
1.6729662841274202
1.4622662479183186
1.3807922155386871
1.008190679505792
1.1531214120851325
1.1262818193836341
1.4466582235507777
1.1303342226232467
0.9649590181542803
0.8323603399347261
0.8960107510176443
0.8374650593117661
0.9061161355125241
0.7186283713012348
1.1731831779505824
0.9061936537765394
1.0110537113284952
0.8575552570169694
0.8441954509456265
0.754447088045348
0.6167214224952372
0.8445818774976304
0.7351480795795365
0.8428386092754786
0.781596492940466
0.6619806815638046
0.6142752123770034
0.8108723996076777
0.792613634676253
0.850594077951545
0.4988493531556901
0.6223488927008569
0.6764363047222964
0.7534674169104328
0.7435727579903633
0.6987310576900213
0.6572000497582282
0.9694782988077822
0.7748910844238805
0.8743430863387671
0.57

In [127]:
x_test = x_test[:1000].reshape(-1, 28*28)
x_test = preparing_data(x_test)

In [128]:
y_test = y_test[:1000].reshape(-1,1)

In [129]:
output = nn.forward(x_test)
prediction = nn.softmax(output)

In [130]:
prediction = np.argmax(prediction, axis=1)

In [134]:
prediction = prediction.reshape(-1,1)

In [141]:
count_of_rigth = np.sum(prediction == y_test)

In [142]:
accuracy = count_of_rigth / y_test.shape[0]
accuracy

0.868